In [ ]:
import os

import pandas as pd

from IPython.display import Markdown

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)

In [ ]:
cwd = os.getcwd()

ss_base_path = os.path.join(cwd, "SingleSpeed", "base", "eplusout.csv")
ss_mod_path = os.path.join(cwd, "SingleSpeed", "mod", "eplusout.csv")

In [ ]:
df_ss_base = pd.read_csv(ss_base_path)
df_ss_base.drop(labels="Date/Time", axis=1, inplace=True)
df_ss_base.head(1)

In [ ]:
df_ss_mod = pd.read_csv(ss_mod_path)
df_ss_mod.drop(labels="Date/Time", axis=1, inplace=True)
df_ss_mod.head(1)

In [ ]:
df_ss_mod.columns

In [ ]:
def calc_heat_reclaimed_frac(df, clg_load, elect, recl):
    return df[recl] / (df[clg_load] + df[elect])

In [ ]:
df_ss_base["Heat Reclaimed [J]"] = calc_heat_reclaimed_frac(df_ss_base, 
                                                            'ASHP CLG COIL:Cooling Coil Total Cooling Energy [J](Hourly)',
                                                            'ASHP CLG COIL:Cooling Coil Electric Energy [J](Hourly)',
                                                            'DESUPERHEATER:Water Heater Heating Energy [J](Hourly)')

df_ss_mod["Heat Reclaimed [J]"] = calc_heat_reclaimed_frac(df_ss_mod, 
                                                            'ASHP CLG COIL:Cooling Coil Total Cooling Energy [J](Hourly)',
                                                            'ASHP CLG COIL:Cooling Coil Electric Energy [J](Hourly)',
                                                            'DESUPERHEATER:Water Heater Heating Energy [J](Hourly)')

In [ ]:
low_idx = 4200
high_idx = 4800

ax = df_ss_base.iloc[low_idx:high_idx].plot(y="Heat Reclaimed [J]", label="develop")
df_ss_mod.iloc[low_idx:high_idx].plot(y="Heat Reclaimed [J]", ax=ax, label="mod")
plt.title("Heat Reclaimed Frac [-]")
ax.set_xlabel("Hour")

In [ ]:
s1 = df_ss_base['WATER HEATER:Water Heater Source Side Heat Transfer Energy [J](Hourly)'].iloc[low_idx:high_idx]
s2 = df_ss_base['DESUPERHEATER:Water Heater Heating Energy [J](Hourly)'].iloc[low_idx:high_idx]

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
ax1.plot(s1, label="WH Src Side Energy [J]")
ax1.plot(s2, label="Desup Energy [J]")
ax1.set_ylabel("Energy [J]")
ax1.legend()

diff = s1 - s2

ax2.plot(diff)
ax2.set_xlabel("Hour")
ax2.set_ylabel("Delta")

plt.suptitle("develop branch")
plt.show()

In [ ]:
sum1 = s1.values.sum()
sum2 = s2.values.sum()
delta_sum = sum1 - sum2
frac = (sum2 - sum1) / sum1 * 100

Markdown("""
## Annual Energy Usage
| WH Src Side Energy [J] | Desup Energy [J] | Delta [J] | Error [%]
|-----------|---------|---------|---------|
| {:.4E} | {:.4E} | {:.4E} | {:0.2f}
""".format(sum1, sum2, delta_sum, frac))

In [ ]:
s1 = df_ss_mod['WATER HEATER:Water Heater Source Side Heat Transfer Energy [J](Hourly)'].iloc[low_idx:high_idx]
s2 = df_ss_mod['DESUPERHEATER:Water Heater Heating Energy [J](Hourly)'].iloc[low_idx:high_idx]

fig, (ax1, ax2) = plt.subplots(2, sharex=True)

ax1.plot(s1, label="WH Src Side Energy [J]")
ax1.plot(s2, label="Desup Energy [J]")
ax1.set_ylabel("Energy [J]")
ax1.legend()

diff = s1 - s2

ax2.plot(diff)
ax2.set_xlabel("Hour")
ax2.set_ylabel("Delta")

plt.suptitle("mod branch")
plt.show()

In [ ]:
sum1 = s1.values.sum()
sum2 = s2.values.sum()
delta_sum = sum1 - sum2
frac = (sum2 - sum1) / sum1 * 100

Markdown("""
## Annual Energy Usage
| WH Src Side Energy [J] | Desup Energy [J] | Delta [J] | Error [%]
|-----------|---------|---------|---------|
| {:.4E} | {:.4E} | {:.4E} | {:0.2f}
""".format(sum1, sum2, delta_sum, frac))

In [ ]:
def plot_all_matches(df1, df2, low_idx=None, high_idx=None):
    for col in df1.columns.values:
        
        s1 = None
        s2 = None
        
        if not low_idx and not high_idx:
            s1 = df1[col]
            s2 = df2[col]
        else:
            s1 = df1[col].iloc[low_idx:high_idx]
            s2 = df2[col].iloc[low_idx:high_idx]
        
        fig, (ax1, ax2) = plt.subplots(2, sharex=True)
        
        ax1.plot(s1, label="develop")
        ax1.plot(s2, label="mod")
        ax1.legend()
        
        ax2.plot(s1 - s2)
        
        ax2.set_ylabel("Delta")
        ax2.set_xlabel("Hour")
        
        plt.suptitle(col)
        plt.show()

In [ ]:
plot_all_matches(df_ss_base, df_ss_mod, low_idx=4200, high_idx=4800)